In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# data : tokyo_covid19_patients.csv

In [ ]:
data0 = pd.read_csv("../input/covid19-tokyo/tokyo_covid19_patients - 0814.csv")
data0[-5:]

In [ ]:
data0['pcr_positives']=1

In [ ]:
data1=data0.groupby('date',as_index=False).sum()
data1[-5:]

In [ ]:
data1['positives mean 7-day']=data1['pcr_positives'].rolling(window=7).mean()
data1['positives max 7-day']=data1['pcr_positives'].rolling(window=7).max()
data1['positives min 7-day']=data1['pcr_positives'].rolling(window=7).min()
data1[-5:].T

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data1['date'],y=data1['positives mean 7-day'],name='positives mean 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data1['date'],y=data1['positives max 7-day'],name='positives max 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data1['date'],y=data1['positives min 7-day'],name='positives min 7-day'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Examined Positives (rolling 7-day) in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases",secondary_y=False)
fig.show()

In [ ]:
col0=data1.columns.to_list()
col1=col0+['pm-7','slope']
data2=pd.DataFrame(columns=col1)
data2[col0]=data1

In [ ]:
n=len(data1)
for  i in range(n):    
    pmi=data2['positives mean 7-day'][i]
    data2.loc[i+7,'pm-7']=pmi

In [ ]:
# Slope value and Rt (simplified effective reproduction number) were defined as follows.
# 'pm-7' means 'positives mean 7-day' value 7 days before.

data2['slope']=(data2['positives mean 7-day']-data2['pm-7'])/7
data２['Rt']=data2['positives mean 7-day']/data2['pm-7']  
data3=data2[['date','pcr_positives','positives mean 7-day','slope','Rt']]
data4=data3[14:-7]

print(data4[243:258]) ### 3rd wave
print()
print(data4[372:387]) ### 4th wave
print()
print(data4[468:483]) ### 5th wave
print()
print(data4[-14:])    ### latest

# A slope value is a sensitive benchmark for an occurence of infectious explosion in the near future.


* On 2020-11-08, the slope value exceeded 3.0. This was the begginning of the 3th wave.
* On 2021-03-14, the slope value exceeded 3.0. This was the begginning of the 4th wave.
* On 2021-06-22, the slope value exceeded 3.0. This was the begginning of the 5th wave.

# The peak of 5th wave has gone? 
* The slope value has reached more than 20 since 2021/07/08.
* The slope value has reached more than 40 since 2021/07/17.
* The slope value has reached more than 50 since 2021/07/25.
* The slope value has reached more than 100 since 2021/07/29.

In [ ]:
print('The peak of 3rd wave') ### more than 100 for 5 days
print(data4[310:317]) ### 3rd wave, MAX 134 on 2021-01-10

In [ ]:
print('The peak of 5th wave') ### more than 100 for 12 days
print(data4[511:525]) ### 5th wave, MAX 237 on 2021-08-02

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data4['date'][280:],y=data4['Rt'][280:],name='Rt'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Rt (simplified effective reproduction number) change in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data4['date'][280:],y=data4['slope'][280:],name='slope'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope change of positives mean 7-day in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data4['date'],y=data4['slope'],name='slope'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data4['date'],y=data4['positives mean 7-day'],name='positives mean 7-day'),secondary_y='positives mean 7-day')
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope change of positives mean 7-day in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()